## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [1]:
import chassisml
import os
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
import numpy as np
import getpass
import json
import time

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

docker hub username········
docker hub password········


## Train model

In [3]:
digits = datasets.load_digits()
data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)

SVC(gamma=0.001)

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [4]:
def process(input_bytes):
    inputs = np.array(json.loads(input_bytes))/2
    inference_results = clf.predict(inputs)
    structured_results = []
    for inference_result in inference_results:
        structured_output = {
            "data": {
                "result": {"classPredictions": [{"class": str(inference_result), "score": str(1)}]}
            }
        }
        structured_results.append(structured_output)
    return structured_results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [5]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com")

## Create and test Chassis model
* Requires `process_fn` defined above

In [6]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/input_sample.json'
results = chassis_model.test(sample_filepath)
print(results)

b'[{"data":{"result":{"classPredictions":[{"class":"4","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"4","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}}]'


In [7]:
env = {
    "name": "sklearn-chassis",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.8.5",
        {
            "pip": [
                "scikit-learn",
                "numpy",
                "chassisml"
            ] 
        }
    ]
}

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [8]:
MODEL_NAME = "Sklearn SVM Digits Classification"
start_time = time.time()
response = chassis_model.publish(
    model_name=MODEL_NAME,
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    conda_env=env
)

Starting build job... Ok!


In [9]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
end_time = time.time()
if final_status['status']['succeeded'] == 1:
    print("Job Completed in {} minutes.\n\nView your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(round((end_time-start_time)/60, 5), dockerhub_user, "-".join(MODEL_NAME.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))

Job Completed in 4.37603 minutes.

View your new container image here: https://hub.docker.com/repository/docker/bmunday131/sklearn-svm-digits-classification
